# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [3]:
from tensorflow import keras
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras import initializers

In [4]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [5]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17465344/17464789 [==============================] - 31s 2us/step


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [6]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [7]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [8]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [9]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [10]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [11]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 11ms/step - loss: 0.6673 - accuracy: 0.6006 - val_loss: 0.6269 - val_accuracy: 0.6881
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.5830 - accuracy: 0.7144 - val_loss: 0.5636 - val_accuracy: 0.7151
Epoch 3/10
782/782 [==============================] - 8s 11ms/step - loss: 0.5172 - accuracy: 0.7531 - val_loss: 0.5308 - val_accuracy: 0.7341
Epoch 4/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4757 - accuracy: 0.7776 - val_loss: 0.5097 - val_accuracy: 0.7484
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4449 - accuracy: 0.7960 - val_loss: 0.4887 - val_accuracy: 0.7560
Epoch 6/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4190 - accuracy: 0.8107 - val_loss: 0.4785 - val_accuracy: 0.7638
Epoch 7/10
782/782 [==============================] - 8s 11ms/step - loss: 0.3992 - accuracy: 0.8216 - val_loss: 0.4597 - val_accuracy: 0.778

In [12]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 2ms/step - loss: 0.4495 - accuracy: 0.7863
Test score: 0.4494694471359253
Test accuracy: 0.7862799763679504


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [13]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant

In [15]:
rnn_hidden_dim = 5
word_embedding_dim = 70
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [16]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

C:\Users\AlexB\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [17]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 19s 24ms/step - loss: 0.6033 - accuracy: 0.6728 - val_loss: 0.5277 - val_accuracy: 0.7363
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.4591 - accuracy: 0.7853 - val_loss: 0.4649 - val_accuracy: 0.7742
Epoch 3/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3957 - accuracy: 0.8236 - val_loss: 0.4291 - val_accuracy: 0.8002
Epoch 4/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3546 - accuracy: 0.8472 - val_loss: 0.4072 - val_accuracy: 0.8120
Epoch 5/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3242 - accuracy: 0.8637 - val_loss: 0.3925 - val_accuracy: 0.8240
Epoch 6/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3024 - accuracy: 0.8732 - val_loss: 0.3967 - val_accuracy: 0.8248
Epoch 7/10
782/782 [==============================] - 18s 23ms/step - loss: 0.2851 - accuracy: 0.8833 - val_loss: 0.3767 - val_accuracy:

In [18]:
model_rnn.evaluate(x_test,y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3657 - accuracy: 0.8392


[0.3657018840312958, 0.8392000198364258]

In [19]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [20]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [21]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [22]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 17s 20ms/step - loss: 0.6788 - accuracy: 0.5346 - val_loss: 0.6533 - val_accuracy: 0.5796
Epoch 2/10
782/782 [==============================] - 15s 20ms/step - loss: 0.6114 - accuracy: 0.6898 - val_loss: 0.5929 - val_accuracy: 0.7540
Epoch 3/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5711 - accuracy: 0.7678 - val_loss: 0.5775 - val_accuracy: 0.7555
Epoch 4/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5504 - accuracy: 0.7940 - val_loss: 0.5615 - val_accuracy: 0.7785
Epoch 5/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5336 - accuracy: 0.8056 - val_loss: 0.5578 - val_accuracy: 0.7778
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5194 - accuracy: 0.8109 - val_loss: 0.5416 - val_accuracy: 0.7855
Epoch 7/10
782/782 [==============================] - 15s 20ms/step - loss: 0.5065 - accuracy: 0.8178 - val_loss: 0.5381 - val_accuracy:

In [23]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 16s 20ms/step - loss: 0.4644 - accuracy: 0.8314 - val_loss: 0.5068 - val_accuracy: 0.7905
Epoch 2/10
782/782 [==============================] - 15s 20ms/step - loss: 0.4552 - accuracy: 0.8329 - val_loss: 0.4993 - val_accuracy: 0.7927
Epoch 3/10
782/782 [==============================] - 15s 20ms/step - loss: 0.4468 - accuracy: 0.8338 - val_loss: 0.4956 - val_accuracy: 0.7912
Epoch 4/10
782/782 [==============================] - 15s 20ms/step - loss: 0.4387 - accuracy: 0.8369 - val_loss: 0.4877 - val_accuracy: 0.7932
Epoch 5/10
782/782 [==============================] - 15s 20ms/step - loss: 0.4310 - accuracy: 0.8395 - val_loss: 0.4926 - val_accuracy: 0.7909
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.4246 - accuracy: 0.8411 - val_loss: 0.4808 - val_accuracy: 0.7960
Epoch 7/10
782/782 [==============================] - 15s 20ms/step - loss: 0.3969 - accuracy: 0.8361 - val_loss: 0.4443 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation